<a href="https://colab.research.google.com/github/mrromaniuc/quantum-computing/blob/main/IBMCertifiedAssociateDeveloperQiskit/5.3-Measure_Fidelity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-visualization
!pip install qiskit-ibm-provider
!pip install imgaug==0.2.5
!pip install pylatexenc

References:
* https://qiskit.org/documentation/tutorials/circuits_advanced/02_operators_overview.html
* https://youtu.be/KCPuw7UDNag
*


In [ ]:
#Setting configuration to use matplotlib in drawings
!mkdir ~/.qiskit
!echo "[default]" > ~/.qiskit/settings.conf
!echo "circuit_drawer = mpl" >> ~/.qiskit/settings.conf
!more ~/.qiskit/settings.conf


#
# Loading IBM credentials
#
API_TOKEN_DIR="/content/drive/'My Drive'/'Colab Notebooks'/QC"
API_TOKEN_FILE="MY_API_TOKEN.txt"

#Mount the google drive first
from google.colab import drive
drive.mount('/content/drive')
#%cd /content/drive/'My Drive'/'Colab Notebooks'/QC
%cd $API_TOKEN_DIR

#Read the MY_API_TOKEN from file...
from qiskit_ibm_provider import IBMProvider
MY_API_TOKEN = ""
with open(API_TOKEN_FILE) as f:
  MY_API_TOKEN=f.readline()
IBMProvider.save_account(MY_API_TOKEN, overwrite=True)

[default]
circuit_drawer = mpl
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/QC


## Measure Fidelity - Process Fidelity
This is a way to measure how close two quantum channels are to each other

It does NOT depend on global phases

In [3]:
#Note that using the comparison, two Operators are not the same if they differ
#by global phase

import numpy as np
from qiskit.quantum_info.operators import Operator

id_matrix = np.array([[1,0],[0,1]])
op1 = Operator(id_matrix)
print(f"Operator 1:\n{op1}")

op2 = -1j*op1
print(f"\nOperator 2:\n{op2}")

print(f"\nIs operator 1 same as 2: {op1 == op2}")

Operator 1:
Operator([[1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j]],
         input_dims=(2,), output_dims=(2,))

Operator 2:
Operator([[0.-1.j, 0.-0.j],
          [0.-0.j, 0.-1.j]],
         input_dims=(2,), output_dims=(2,))

Is operator 1 same as 2: False


In [5]:
#using process fidelity, the operators are considered the same if they differ
#only by a global phase

import numpy as np
from qiskit.quantum_info import process_fidelity
from qiskit.quantum_info.operators import Operator

xpauli_matrix= np.array([[0,1],[1,0]])
op1 = Operator(xpauli_matrix)
print(f"Operator 1:\n{op1}")

op2 = np.exp(1j*0.5)*op1
print(f"\nOperator 2:\n{op2}")

print(f"\nProcess fidelity = {process_fidelity(op1, op2)}")

Operator 1:
Operator([[0.+0.j, 1.+0.j],
          [1.+0.j, 0.+0.j]],
         input_dims=(2,), output_dims=(2,))

Operator 2:
Operator([[0.        +0.j        , 0.87758256+0.47942554j],
          [0.87758256+0.47942554j, 0.        +0.j        ]],
         input_dims=(2,), output_dims=(2,))
Process fidelity = 1.0
